In [ ]:
!wget -O - --no-check-certificate https://hyeshik.qbio.io/binfo/binfo1-datapack1.tar | tar -C /mnt/d/binfodata -xf -
!wget --no-check-certificate -O /mnt/d/binfodata/binfo1-datapack1/gencode.gtf.gz http://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M27/gencode.vM27.annotation.gtf.gz
!md5sum -c /mnt/d/binfodata/binfo1-datapack1/*
!ls -al /mnt/d/binfodata/binfo*

In [ ]:
!mkdir -p /mnt/d/binfodata/binfo1-work/
%cd /mnt/d/binfodata/binfo1-work/
!cp ../binfo1-datapack1/* .

In [ ]:
!conda install -y bedtools bioawk samtools subread

In [ ]:
!pip install pysam

압축풀기(reference genome)

In [ ]:
!gunzip /mnt/d/binfodata/binfo1-datapack1/mm9.fa.gz

Directory 지정

In [2]:
%cd /mnt/d/binfodata/binfo1-work

/mnt/d/binfodata/binfo1-work


Samtools를 이용해 bam 파일의 unmapped data를 제거합니다.

In [ ]:
!(samtools view -H ../binfo1-datapack1/CLIP-35L33G.bam; \
  samtools view -F 4 ../binfo1-datapack1/CLIP-35L33G.bam) | \
 samtools view -b -o filtered-CLIP-35L33G.bam

In [ ]:
!samtools mpileup filtered-CLIP-35L33G.bam > filtered-CLIP-35L33G.pileup

In [ ]:
!head filtered-CLIP-35L33G-pileup.bam

이거부터 돌려보자.

In [ ]:
import pandas as pd

# Step 1: 파일업 결과를 읽고, read count 50 이하인 부분 제거
pileup_file = 'filtered-CLIP-35L33G.pileup'
filtered_pileup_file = 'filtered_pileup.txt'

with open(pileup_file, 'r') as infile, open(filtered_pileup_file, 'w') as outfile:
    for line in infile:
        chrom, pos, ref, depth, bases, qual = line.strip().split('\t')[:6]
        depth = int(depth)
        if depth > 50:
            outfile.write(line)

print(f"Filtered pileup data saved to: {filtered_pileup_file}")

In [ ]:
!tail filtered_pileup.txt

커널 용량 에러가 끊임없이 발생하여 청크 100개로 나눔.

In [3]:
import os

input_file = 'filtered_pileup.txt'
num_parts = 100  # 더 작은 청크로 나누기

# 파일의 총 라인 수를 계산합니다.
total_lines = sum(1 for line in open(input_file))

# 각 파일에 들어갈 라인의 수를 계산합니다.
lines_per_file = total_lines // num_parts

with open(input_file, 'r') as f:
    for i in range(num_parts):
        with open(f'filtered_pileup_part_{i+1}.txt', 'w') as out_file:
            for j in range(lines_per_file):
                line = f.readline()
                if not line:
                    break
                out_file.write(line)
                
            # 마지막 파일에 남은 라인을 추가합니다.
            if i == num_parts - 1:
                for line in f:
                    out_file.write(line)

print(f"{num_parts}개의 작은 파일로 나눔 완료.")

100개의 작은 파일로 나눔 완료.


In [5]:
import pandas as pd
import numpy as np

# Shannon entropy 계산 함수 (벡터화된 버전)
def calculate_shannon_entropy_vec(counts):
    total = np.sum(counts, axis=1)
    probs = np.where(total[:, None] == 0, 0, counts / total[:, None])
    entropy = -np.sum(probs * np.log2(probs, where=(probs > 0)), axis=1)
    return entropy

output_file = 'binding_sites.csv'
cutoff = 0.8
num_parts = 100

# 결과를 저장할 리스트 초기화
results = []

# 각 작은 파일을 처리
for i in range(num_parts):
    part_file = f'filtered_pileup_part_{i+1}.txt'
    
    # 파일을 청크 단위로 읽기
    chunk = pd.read_csv(part_file, sep='\t', header=None, names=['chrom', 'pos', 'ref', 'depth', 'bases', 'qual'], dtype={'chrom': str, 'pos': int, 'ref': str, 'depth': int, 'bases': str, 'qual': str})
    
    # 각 염기 빈도 계산
    chunk['count_A'] = chunk['bases'].str.count('A')
    chunk['count_C'] = chunk['bases'].str.count('C')
    chunk['count_G'] = chunk['bases'].str.count('G')
    chunk['count_T'] = chunk['bases'].str.count('T')
    
    # Shannon entropy 계산
    counts = chunk[['count_A', 'count_C', 'count_G', 'count_T']].values
    chunk['entropy'] = calculate_shannon_entropy_vec(counts)
    
    # Shannon entropy 값이 cutoff를 통과하는 binding site 선택
    filtered_chunk = chunk[chunk['entropy'] > cutoff]
    results.append(filtered_chunk)

# 결과를 데이터프레임으로 병합
binding_sites = pd.concat(results, ignore_index=True)

# 결과를 CSV 파일로 저장
binding_sites.to_csv(output_file, index=False)

print(f"Binding sites saved to: {output_file}")

/tmp/ipykernel_181529/2421546399.py:7: RuntimeWarning: invalid value encountered in divide
  probs = np.where(total[:, None] == 0, 0, counts / total[:, None])
/tmp/ipykernel_181529/2421546399.py:7: RuntimeWarning: invalid value encountered in divide
  probs = np.where(total[:, None] == 0, 0, counts / total[:, None])
/tmp/ipykernel_181529/2421546399.py:7: RuntimeWarning: invalid value encountered in divide
  probs = np.where(total[:, None] == 0, 0, counts / total[:, None])
/tmp/ipykernel_181529/2421546399.py:7: RuntimeWarning: invalid value encountered in divide
  probs = np.where(total[:, None] == 0, 0, counts / total[:, None])
/tmp/ipykernel_181529/2421546399.py:7: RuntimeWarning: invalid value encountered in divide
  probs = np.where(total[:, None] == 0, 0, counts / total[:, None])
/tmp/ipykernel_181529/2421546399.py:7: RuntimeWarning: invalid value encountered in divide
  probs = np.where(total[:, None] == 0, 0, counts / total[:, None])
/tmp/ipykernel_181529/2421546399.py:7: Runtime

Binding sites saved to: binding_sites.csv


In [ ]:
!samtools index filtered-CLIP-35L33G.bam

In [8]:
import pandas as pd
import pysam

# 파일 경로 설정
binding_sites_file = 'binding_sites.csv'
bam_file = 'filtered-CLIP-35L33G.bam'
weblogo_input_file = 'weblogo_input.txt'

# Binding sites 파일 읽기
binding_sites = pd.read_csv(binding_sites_file)

# BAM 파일 열기
bam = pysam.AlignmentFile(bam_file, "rb")

# RNA 시퀀스 변환 함수
def dna_to_rna(dna_seq):
    translation_table = str.maketrans('ACGT', 'ACGU')
    return dna_seq.translate(translation_table)

# Flanking residues 추출
with open(weblogo_input_file, 'w') as weblogo_out:
    for _, row in binding_sites.iterrows():
        chrom = row['chrom']
        pos = row['pos']
        start = pos - 10
        end = pos + 10
        
        sequences = []
        
        for read in bam.fetch(chrom, start, end):
            if not read.is_unmapped:
                read_start = read.reference_start
                read_end = read.reference_end
                if read_start <= pos < read_end:  # ensure pos is within read
                    seq = read.query_sequence
                    seq_start = max(0, pos - read_start - 10)
                    seq_end = min(len(seq), pos - read_start + 11)  # inclusive of the position
                    flanking_seq = seq[seq_start:seq_end]
                    rna_seq = dna_to_rna(flanking_seq)
                    
                    # Calculate padding
                    left_padding_needed = max(0, 10 - (pos - read_start))
                    right_padding_needed = max(0, 10 - (read_end - pos - 1))
                    
                    left_padding = '-' * left_padding_needed
                    right_padding = '-' * right_padding_needed

                    padded_seq = left_padding + rna_seq + right_padding

                    if len(padded_seq) < 21:
                        padded_seq = padded_seq + '-' * (21 - len(padded_seq))
                    padded_seq = padded_seq[:21]

                    sequences.append(padded_seq)

        if sequences:
            for seq in sequences:
                weblogo_out.write(seq + '\n')
        else:
            # No reads found, add '-' padding only
            padded_seq = '-' * 21
            weblogo_out.write(padded_seq + '\n')

print(f"WebLogo input file saved to: {weblogo_input_file}")


WebLogo input file saved to: weblogo_input.txt


In [9]:
!pip install weblogo

^C
ERROR: Operation cancelled by user


Weblogo로 하려고 많은 노력을 기울였으나 Logomaker로 만드는 것이 용량상 합리적.

In [21]:
!pip install logomaker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 15.0 MB/s eta 0:00:0000:0100:01


In [ ]:
!sudo apt-get update
!sudo apt-get install ghostscript

In [19]:
import pandas as pd
import logomaker
import matplotlib.pyplot as plt

# 파일 경로 설정
input_file = 'weblogo_input.txt'
output_image_bits = 'weblogo_output_bits.png'
output_image_prob = 'weblogo_output_prob.png'

positions = list(range(-10, 11))
annotate = ['-10', '', '', '', '', '-5', '', '', '', '', '0', '', '', '', '', '5', '', '', '', '10']

# annotate 리스트를 positions 리스트와 같은 길이로 맞추기 위해 빈 문자열 추가
if len(annotate) < len(positions):
    annotate += [''] * (len(positions) - len(annotate))

# 시퀀스 데이터 로드
sequences = []
with open(input_file, 'r') as f:
    for line in f:
        sequences.append(line.strip())

# 시퀀스를 데이터프레임으로 변환
data = {'sequence': sequences}
df = pd.DataFrame(data)

# 시퀀스를 logomaker에 맞는 형식으로 변환
logo_df = logomaker.alignment_to_matrix(df['sequence'], to_type='information')
probability_df = logomaker.alignment_to_matrix(df['sequence'], to_type='probability')

# 염기 색상 설정
custom_color_scheme = {
    'A': '#1f77b4',  # 파랑
    'C': '#2ca02c',  # 초록
    'G': '#ff7f0e',  # 노랑
    'U': '#d62728',  # 빨강 (T 대신 U 사용)
    'N': '#000000'   # 검정색
}

# Information Content 로고 생성
plt.figure(figsize=(15, 5))
logo = logomaker.Logo(logo_df, color_scheme=custom_color_scheme)
logo.style_spines(visible=False)
logo.style_spines(spines=['left', 'bottom'], visible=True)
logo.ax.set_ylabel('Bits')
logo.ax.set_xlabel('Distance from cross-linked nucleotide')
logo.ax.set_xticks(list(range(len(positions))))
logo.ax.set_xticklabels(annotate)
logo.ax.set_title('Information Content')
plt.savefig(output_image_bits)
plt.close()

# Base Probability 로고 생성
plt.figure(figsize=(15, 5))
logo = logomaker.Logo(probability_df, color_scheme=custom_color_scheme)
logo.style_spines(visible=False)
logo.style_spines(spines=['left', 'bottom'], visible=True)
logo.ax.set_ylabel('Probability')
logo.ax.set_xlabel('Distance from cross-linked nucleotide')
logo.ax.set_xticks(list(range(len(positions))))
logo.ax.set_xticklabels(annotate)
logo.ax.set_title('Base Probability')
plt.savefig(output_image_prob)
plt.close()

print("WebLogo 이미지 생성 완료.")

: 

2개로 쪼개야 Kernerl 문제 사라짐.

In [15]:
import os

input_file = 'weblogo_input.txt'
output_dir = 'weblogo_chunks'
num_parts = 2  # 나눌 파일 개수 설정

# 출력 디렉토리 생성
os.makedirs(output_dir, exist_ok=True)

# 파일의 총 라인 수를 계산합니다.
total_lines = sum(1 for line in open(input_file))

# 각 파일에 들어갈 라인의 수를 계산합니다.
lines_per_file = total_lines // num_parts

with open(input_file, 'r') as f:
    for i in range(num_parts):
        output_path = os.path.join(output_dir, f'weblogo_input_part_{i+1}.txt')
        with open(output_path, 'w') as out_file:
            for j in range(lines_per_file):
                line = f.readline()
                if not line:
                    break
                out_file.write(line)
                
    # 마지막 파일에 남은 라인을 추가합니다.
    # 이 블록을 밖으로 이동하여 모든 라인을 파일에 기록합니다.
    if i == num_parts - 1:
        with open(output_path, 'a') as out_file:
            for line in f:
                out_file.write(line)

print(f"{num_parts}개의 작은 파일로 나눔 완료.")

2개의 작은 파일로 나눔 완료.


In [5]:
import os
import pandas as pd
import logomaker
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

output_dir = 'weblogo_chunks'
num_parts = 2  # 위에서 설정한 파일 개수와 동일해야 함

positions = list(range(-10, 11))
annotate = ['-10', '', '', '', '', '-5', '', '', '', '', '0', '', '', '', '', '5', '', '', '', '10']

# annotate 리스트를 positions 리스트와 같은 길이로 맞추기 위해 빈 문자열 추가
if len(annotate) < len(positions):
    annotate += [''] * (len(positions) - len(annotate))

# 염기 색상 설정
custom_color_scheme = {
    'A': '#1f77b4',  # 파랑
    'C': '#2ca02c',  # 초록
    'G': '#ff7f0e',  # 노랑
    'U': '#d62728',  # 빨강 (T 대신 U 사용)
    'N': '#000000'   # 검정색
}

for i in range(num_parts):
    part_file = os.path.join(output_dir, f'weblogo_input_part_{i+1}.txt')
    output_image_bits = os.path.join(output_dir, f'weblogo_output_part_{i+1}_bits.png')
    output_image_prob = os.path.join(output_dir, f'weblogo_output_part_{i+1}_prob.png')
    
    # 시퀀스 데이터 로드
    sequences = []
    with open(part_file, 'r') as f:
        for line in f:
            sequences.append(line.strip())
    
    # 시퀀스를 데이터프레임으로 변환
    data = {'sequence': sequences}
    df = pd.DataFrame(data)
    
    # 시퀀스를 logomaker에 맞는 형식으로 변환
    logo_df = logomaker.alignment_to_matrix(df['sequence'], to_type='information')
    probability_df = logomaker.alignment_to_matrix(df['sequence'], to_type='probability')
    
    # Information Content 로고 생성
    plt.figure(figsize=(15, 5))
    logo = logomaker.Logo(logo_df, color_scheme=custom_color_scheme)
    logo.style_spines(visible=False)
    logo.style_spines(spines=['left', 'bottom'], visible=True)
    logo.ax.set_ylabel('Bits')
    logo.ax.set_xlabel('Distance from cross-linked nucleotide')
    logo.ax.set_xticks(list(range(len(positions))))
    logo.ax.set_xticklabels(annotate)
    logo.ax.set_title(f'Information Content Part {i+1}')
    plt.savefig(output_image_bits)
    plt.close()
    
    # Base Probability 로고 생성
    plt.figure(figsize=(15, 5))
    logo = logomaker.Logo(probability_df, color_scheme=custom_color_scheme)
    logo.style_spines(visible=False)
    logo.style_spines(spines=['left', 'bottom'], visible=True)
    logo.ax.set_ylabel('Frequency')
    logo.ax.set_xlabel('Distance from cross-linked nucleotide')
    logo.ax.set_xticks(list(range(len(positions))))
    logo.ax.set_xticklabels(annotate)
    logo.ax.set_title(f'Base Probability Part {i+1}')
    plt.savefig(output_image_prob)
    plt.close()

print("WebLogo 이미지 생성 완료.")

# 이미지 병합
def merge_images_with_labels(image_paths, output_path, x_labels, y_labels):
    images = [Image.open(x) for x in image_paths]
    widths, heights = zip(*(i.size for i in images))
    
    total_height = sum(heights)
    max_width = max(widths)
    
    # 새로운 이미지 생성 (라벨 공간 추가)
    label_height = 50
    combined_image = Image.new('RGB', (max_width, total_height + label_height))
    draw = ImageDraw.Draw(combined_image)
    
    # 폰트 설정
    try:
        font = ImageFont.truetype("arial.ttf", 20)
    except IOError:
        font = ImageFont.load_default()
    
    y_offset = 0
    for im in images:
        combined_image.paste(im, (0, y_offset))
        y_offset += im.size[1]
    
    # x축 라벨 추가
    x_offset = 10
    for label in x_labels:
        draw.text((x_offset, total_height), label, font=font, fill='black')
        x_offset += (max_width // len(x_labels))
    
    # y축 라벨 추가
    y_offset = 10
    for label in y_labels:
        draw.text((0, y_offset), label, font=font, fill='black')
        y_offset += (total_height // len(y_labels))
    
    combined_image.save(output_path)

# 병합할 이미지 파일 경로 설정
bits_images = [os.path.join(output_dir, f'weblogo_output_part_{i+1}_bits.png') for i in range(num_parts)]
prob_images = [os.path.join(output_dir, f'weblogo_output_part_{i+1}_prob.png') for i in range(num_parts)]

# 병합된 이미지 저장 경로 설정
combined_bits_image = 'combined_weblogo_output_bits.png'
combined_prob_image = 'combined_weblogo_output_prob.png'

# x축 라벨 설정
x_labels = annotate

# y축 라벨 설정 (예시로 비워둠, 필요시 추가)
y_labels = [''] * num_parts

# 이미지 병합 (라벨 추가)
merge_images_with_labels(bits_images, combined_bits_image, x_labels, y_labels)
merge_images_with_labels(prob_images, combined_prob_image, x_labels, y_labels)

print("이미지 병합 완료.")


WebLogo 이미지 생성 완료.
이미지 병합 완료.


<Figure size 1500x500 with 0 Axes>

<Figure size 1500x500 with 0 Axes>

<Figure size 1500x500 with 0 Axes>

<Figure size 1500x500 with 0 Axes>

In [10]:
!pip install pillow